$$
    \frac{\partial \phi_i}{\partial t} = - \mathcal{L}_i \cdot
\frac{\delta F}{\delta \phi_i} \quad i = 1, 2, \ldots, N
$$

Here, F represents the total free energy of the system, consisting of bulk energy density f and the gradient energy penalty associated with the interfaces of different grains.

\begin{equation}
    F = \int_V \left[ f(\phi_1, \phi_2, \ldots, \phi_N) + \sum_i^N \frac{\kappa_i}{2} \left| \nabla \phi_i \right|^2 \right] dv
\end{equation}

The bulk energy density function f is modeled using a multi-well potential for each grain order parameter $\phi_i$, penalizing overlaps between grains through the interaction term with coefficient $\gamma$

\begin{equation}
 f(\phi_1, \phi_2, \ldots, \phi_N) = \sum_{i=1}^{N} \left( -\frac{1}{2} \alpha \phi_i^2 + \frac{1}{4} \beta \phi_i^4 \right) + \gamma \sum_{i=1}^{N} \sum_{j \ne i}^{N} \phi_i^2 \phi_j^2   
\end{equation}

Taking the functional derivative of F with respect to gives us the driving force for the evolution of each grain, combining local bulk terms and gradient penalty terms.

\begin{equation}
    \frac{\delta F}{\delta \phi_i} = \frac{\partial f}{\partial \phi_i} -  \kappa_i \nabla^2 \phi_i
\end{equation}

Computing the partial derivative of f with respect to, we obtain contributions from self-interaction and interactions with neighboring grains.

\begin{equation}
   \frac{\partial f}{\partial \phi_i} = - \alpha \phi_i + \beta \phi_i^3 + 2 \gamma \phi_i \sum_{i \ne j}^{N} \phi_j^2
\end{equation}

Substituting this into the expression for $\frac{\delta F}{\delta \phi_i}$, we get the explicit form of the variational derivative governing the evolution of $\phi_i$.

\begin{equation}
    \frac{\delta F}{\delta \phi_i} =  - \alpha \phi_i + \beta \phi_i^3 + 2 \gamma \phi_i \sum_{i \ne j}^{N} \phi_j^2 -\kappa_i \nabla^2 \phi_i
\end{equation}

Finally, substituting into the governing equation (Eq 1), we obtain the time evolution equation for each order parameter which is a multivariate Allen-Cahn type equation.

\begin{equation}
    \frac{\partial \phi_i}{\partial t} = - \mathcal{L}_i \left(- \alpha \phi_i + \beta \phi_i^3 + 2 \gamma \phi_i \sum_{i \ne j}^{N} \phi_j^2 -\kappa_i \nabla^2 \phi_i\right)  \quad i = 1, 2, \ldots, N
\end{equation}



\begin{equation}
    \frac{\partial \phi_i}{\partial t} = - \mathcal{L}_i \frac{\partial f}{\partial \phi_i} + \mathcal{L}_i \kappa_i \nabla^2 \phi_i  \quad i = 1, 2, \ldots, N
\end{equation}

\noindent Applying Fourier transform to both sides of Eq.~\ref{eq:1}, the Laplacian term $\nabla^2 \phi_i$ transforms to $-(2\pi k)^2 \tilde{\phi_i}$ in Fourier space.


\begin{equation}
    \frac{\partial \tilde{\phi_i}}{\partial t} = - \mathcal{L}_i \frac{\partial \tilde{f}}{\partial \phi_i} + (2\pi i k)^2 \mathcal{L}_i \kappa_i \tilde{\phi_i}  
\end{equation}

\noindent Using a forward difference scheme for the time derivative, the discretized form of the equation in Fourier space becomes:

\begin{equation}
    \frac{\tilde{\phi_i}^{n+1} -\tilde{\phi_i}^{n}}{\Delta t} = - \mathcal{L}_i \frac{\partial \tilde{f}^n}{\partial \phi_i} -4 \pi^2 k^2 \mathcal{L}_i \kappa_i \tilde{\phi_i}^{n+1}  
\end{equation}

\noindent Defining $\tilde{g}(\phi_i)$ as the Fourier transform of the bulk free energy derivative evaluated at time step $n$:

\begin{equation}
    \tilde{g}(\phi_i)= \frac{\partial \tilde{f}^n}{\partial \phi_i}
\end{equation}

\noindent Rearranging the terms, the explicit expression for $\tilde{\phi_i}^{n+1}$ in Fourier space is obtained as:

\begin{equation}
    \tilde{\phi_i}^{n+1} =
\frac{\tilde{ \phi_i}^n - \Delta t \mathcal{L} \tilde{g}(\phi_i)^n}{1 + 4 \pi^2 \mathcal{L} \kappa k^2 \Delta t
}
\end{equation}

# Final Spinodal Decomposition

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance

Nx, Ny = 256, 256
num_grains = 100

# Step 1: Place random seed points
np.random.seed(42)
seed_x = np.random.randint(0, Nx, size=num_grains)
seed_y = np.random.randint(0, Ny, size=num_grains)

# Step 2: Create meshgrid of coordinates
X, Y = np.meshgrid(np.arange(Nx), np.arange(Ny), indexing='ij')

# Step 3: Assign each pixel to the nearest seed (grain)
phi = np.zeros((Nx, Ny), dtype=int)
for i in range(Nx):
    for j in range(Ny):
        dists = (seed_x - i)**2 + (seed_y - j)**2
        phi[i, j] = np.argmin(dists)

# Step 4: Visualize
plt.figure(figsize=(6, 6))
plt.imshow(phi, cmap='tab20', origin='lower')
plt.title('Voronoi-style Initial Microstructure')
plt.axis('off')
plt.show()


### With plt.show

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

# Parameters
Nx = Ny = 100
dx = dy = 1.0
dt = 0.1
nstep = 150
print_step = 5
Ngrains = 10

# Model parameters
alpha = 0.5
beta = 2.5
gamma = 1.0
L = 1
kappa = 10.0

# Initial random grain ID assignment
phi = np.random.randint(1, Ngrains + 1, size=(Ny, Nx))

# Initialize eta fields
etas = np.zeros((Ngrains, Ny, Nx))
for g in range(Ngrains):
    etas[g] = (phi == (g + 1)).astype(float)

# Fourier domain variables
kx = np.fft.fftfreq(Nx, d=dx)
ky = np.fft.fftfreq(Ny, d=dy)
KX, KY = np.meshgrid(kx, ky)
kpow2 = KX**2 + KY**2

# Time evolution loop
for step in range(nstep):
    etas_new = np.zeros_like(etas)

    for i in range(Ngrains):
        eta_i = etas[i]
        # Correctly compute interaction term: sum of η_j², j ≠ i
        sum_eta_j_sq = np.sum([etas[j]**2 for j in range(Ngrains) if j != i], axis=0)

        # Free energy derivative with coupling
        g = -alpha * eta_i + beta * eta_i**3 + 2 * gamma * eta_i * sum_eta_j_sq
        g_hat = np.fft.fft2(g)
        eta_hat = np.fft.fft2(eta_i)

        # Fourier space update
        numerator = eta_hat - dt * L * g_hat
        denominator = 1 + 4 * np.pi**2 * dt * L * kappa * kpow2
        eta_i_new = numerator / denominator

        etas_new[i] = np.real(np.fft.ifft2(eta_i_new))

    # Update all eta fields
    etas = np.copy(etas_new)

    # Visualization
    if step % print_step == 0:
        grain_map = np.argmax(etas, axis=0) + 1  # Grain number 1 to Ngrains

        plt.figure(figsize=(6, 5))
        cmap = plt.get_cmap('tab10', Ngrains)
        bounds = np.arange(1, Ngrains + 2)
        norm = mcolors.BoundaryNorm(bounds, cmap.N)

        im = plt.imshow(grain_map, cmap=cmap, norm=norm, origin='lower')
        cbar = plt.colorbar(im, ticks=np.arange(1, Ngrains + 1))
        cbar.set_label('Grain Number')
        plt.title(f'Grain Structure at Step {step}')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.grid(False)
        plt.show()


### Not Our

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft2, ifft2

def GrainGrowth(Nx, Ny, dx, dy, end_time, time_step):
    # Parameters initialization
    halfNx = Nx // 2
    halfNy = Ny // 2
    start_time = 1
    
    # Define the initial profile of the order parameters
    np.random.seed(0)
    phi = np.random.randint(1, 5, size=(Nx, Ny))  # Random initialization of phi between 1 and 10
    
    # Initialize the ten order parameters (eta1 to eta10)
    eta = [np.zeros((Nx, Ny)) for _ in range(5)]
    
    # Initialize b and gets
    b = np.zeros((Nx, Ny))
    
    # Assigning the order parameters value for ten different grain orientations
    for i in range(Nx):
        for j in range(Ny):
            if phi[i, j] == 1:
                eta[0][i, j] = 1
            elif phi[i, j] == 2:
                eta[1][i, j] = 1
            elif phi[i, j] == 3:
                eta[2][i, j] = 1
            elif phi[i, j] == 4:
                eta[3][i, j] = 1
            
    # Time evolution parameters
    dt = 0.1
    delkx = 2 * np.pi / (Nx * dx)
    delky = 2 * np.pi / (Ny * dy)
    A = 1.0
    L = 1.0
    kappa = 1.0
    
    # Temporal evolution loop
    for temp in range(start_time, end_time + 1):
        # Calculate gets for each eta
        gets = np.zeros((4, Nx, Ny))
        for m in range(4):
            for i in range(Nx):
                for j in range(Ny):
                    gets[m, i, j] = -eta[m][i, j] + eta[m][i, j]**3 + 2 * eta[m][i, j] * np.sum(np.fromiter((eta[n][i, j] for n in range(4) if n != m), dtype=float))

        # FFT of the eta fields
        eta_hat = [fft2(eta[m]) for m in range(4)]
        gets_hat = [fft2(gets[m]) for m in range(4)]
        
        # Temporal evolution of eta fields in Fourier space
        for i in range(Nx):
            kx = (i if i <= halfNx else i - Nx) * delkx
            for j in range(Ny):
                ky = (j if j <= halfNy else j - Ny) * delky
                k2 = kx**2 + ky**2
                for m in range(4):
                    eta_hat[m][i, j] = (eta_hat[m][i, j] - L * dt * gets_hat[m][i, j]) / (1 + 2 * L * kappa * k2 * dt)
        
        # Inverse FFT to get updated eta fields
        eta = [np.real(ifft2(eta_hat[m])) for m in range(4)]
        
        # Update b
        for i in range(Nx):
            for j in range(Ny):
                b[i, j] = sum(eta[m][i, j] * np.sum(np.fromiter((eta[n][i, j] for n in range(4) if n != m), dtype=float)) for m in range(4))
        
        # Visualization output
        if temp % time_step == 0:
            plt.figure(figsize=(8, 6))
            plt.imshow(b, cmap='jet', interpolation='none')
            plt.colorbar()
            plt.title(f"Time {temp}")
            plt.axis('off')
            plt.show()

    print("The code execution has finished.")

# Example usage
Nx = 256
Ny = 256
dx = 1.0
dy = 1.0
end_time = 100
time_step = 10
GrainGrowth(Nx, Ny, dx, dy, end_time, time_step)


### Plot eta

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib import colors as mcolors
from IPython.display import display, HTML
import time

# Parameters
Nx = Ny = 256
dx = dy = 1.0
dt = 0.1
nstep = 500
print_step = 5
Ngrains = 20

# Model parameters
alpha = 0.5
beta = 2.5
gamma = 1.0
L = 1
kappa = 10.0

# Set random seed
np.random.seed(0)

# Initialize phi and eta arrays
phi = np.random.randint(Ngrains, size=(Nx, Ny))
eta = {i: np.zeros((Nx, Ny)) for i in range(1, Ngrains + 1)}
for i in range(Nx):
    for j in range(Ny):
        eta[phi[i, j] + 1][i, j] = 1

# Fourier variables
kx = np.fft.fftfreq(Nx, d=dx)
ky = np.fft.fftfreq(Ny, d=dy)
KX, KY = np.meshgrid(kx, ky)
kpow2 = KX**2 + KY**2

# Store grain maps
grain_maps = []

# Start timing the simulation
start_time = time.time()

# Time evolution
for step in range(nstep):
    etas_new = np.zeros((Ngrains, Nx, Ny))
    
    for i in range(1, Ngrains + 1):
        eta_i = eta[i]
        sum_eta_j_sq = np.sum([eta[j]**2 for j in range(1, Ngrains + 1) if j != i], axis=0)

        g = -alpha * eta_i + beta * eta_i**3 + 2 * gamma * eta_i * sum_eta_j_sq
        g_hat = np.fft.fft2(g)
        eta_hat = np.fft.fft2(eta_i)

        numerator = eta_hat - dt * L * g_hat
        denominator = 1 + 4 * np.pi**2 * dt * L * kappa * kpow2
        eta_i_new = numerator / denominator
        etas_new[i-1] = np.real(np.fft.ifft2(eta_i_new))

    # Update eta fields
    for i in range(1, Ngrains + 1):
        eta[i] = etas_new[i - 1]
        
    # Record grain maps
    if step % print_step == 0:
        grain_map = np.argmax(etas_new, axis=0) + 1  # 1-based grain index
        grain_maps.append(grain_map.copy())

# End timing the simulation
end_time = time.time()
total_time = end_time - start_time

# Create animation of grain maps
fig, ax = plt.subplots(figsize=(6, 6))

# Create a colormap with Ngrains distinct colors
cmap = plt.get_cmap('tab20', Ngrains)  # 'tab20' gives us a colormap with 20 distinct colors
norm = mcolors.BoundaryNorm(np.arange(1, Ngrains + 2), cmap.N)  # Ensure we have Ngrains + 1 boundaries

im = ax.imshow(grain_maps[0], cmap=cmap, norm=norm, origin='lower')
cbar = plt.colorbar(im, ticks=np.arange(1, Ngrains + 1), ax=ax)
cbar.set_label('Grain Number')
ax.set_title('Grain Growth Simulation')
ax.set_xlabel('x')
ax.set_ylabel('y')

# Update function with time display
def update(frame):
    im.set_array(grain_maps[frame])
    current_time = frame * print_step * dt  # Calculate current time
    ax.set_title(f"Grain Growth at Time = {current_time:.2f} s (Step {frame * print_step})")
    return [im]

ani = FuncAnimation(fig, update, frames=len(grain_maps), interval=200, blit=True)
plt.close()

# Show animation
display(HTML(ani.to_jshtml()))

# Print total time it took to run the simulation
print(f"Total simulation time: {total_time:.2f} seconds")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib import colors as mcolors
from IPython.display import display, HTML
import time

# Parameters
Nx = Ny = 256
dx = dy = 1.0
dt = 0.1
nstep = 500
print_step = 5
Ngrains = 4

# Model parameters
alpha = 0.5
beta = 2.5
gamma = 1.0
L = 1
kappa = 10.0

# Set random seed
np.random.seed(0)

# Initialize phi and eta arrays
phi = np.random.randint(Ngrains, size=(Nx, Ny))
eta = {i: np.zeros((Nx, Ny)) for i in range(1, Ngrains + 1)}
for i in range(Nx):
    for j in range(Ny):
        eta[phi[i, j] + 1][i, j] = 1

# Fourier variables
kx = np.fft.fftfreq(Nx, d=dx)
ky = np.fft.fftfreq(Ny, d=dy)
KX, KY = np.meshgrid(kx, ky)
kpow2 = KX**2 + KY**2

# Store grain maps
grain_maps = []

# Start timing the simulation
start_time = time.time()

# Time evolution
for step in range(nstep):
    etas_new = np.zeros((Ngrains, Nx, Ny))
    
    for i in range(1, Ngrains + 1):
        eta_i = eta[i]
        sum_eta_j_sq = np.sum([eta[j]**2 for j in range(1, Ngrains + 1) if j != i], axis=0)

        g = -alpha * eta_i + beta * eta_i**3 + 2 * gamma * eta_i * sum_eta_j_sq
        g_hat = np.fft.fft2(g)
        eta_hat = np.fft.fft2(eta_i)

        numerator = eta_hat - dt * L * g_hat
        denominator = 1 + 4 * np.pi**2 * dt * L * kappa * kpow2
        eta_i_new = numerator / denominator
        etas_new[i-1] = np.real(np.fft.ifft2(eta_i_new))

    # Update eta fields
    for i in range(1, Ngrains + 1):
        eta[i] = etas_new[i - 1]
        
    # Record grain maps
    if step % print_step == 0:
        grain_map = np.argmax(etas_new, axis=0) + 1  # 1-based grain index
        grain_maps.append(grain_map.copy())

# End timing the simulation
end_time = time.time()
total_time = end_time - start_time

# Create animation of grain maps
fig, ax = plt.subplots(figsize=(6, 6))
cmap = plt.get_cmap('tab10', Ngrains)
norm = mcolors.BoundaryNorm(np.arange(1, Ngrains + 2), cmap.N)
im = ax.imshow(grain_maps[0], cmap=cmap, norm=norm, origin='lower')
cbar = plt.colorbar(im, ticks=np.arange(1, Ngrains + 1), ax=ax)
cbar.set_label('Grain Number')
ax.set_title('Grain Growth Simulation')
ax.set_xlabel('x')
ax.set_ylabel('y')

# Update function with time display
def update(frame):
    im.set_array(grain_maps[frame])
    current_time = frame * print_step * dt  # Calculate current time
    ax.set_title(f"Grain Growth at Time = {current_time:.2f} s (Step {frame * print_step})")
    return [im]

ani = FuncAnimation(fig, update, frames=len(grain_maps), interval=200, blit=True)
plt.close()

# Show animation
display(HTML(ani.to_jshtml()))

# Print total time it took to run the simulation
print(f"Total simulation time: {total_time:.2f} seconds")


### Plot interation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

# Parameters
Nx = Ny = 256
dx = dy = 1.0
dt = 0.1
nstep = 500
print_step = 10
Ngrains = 10

# Model parameters
alpha = 1
beta = 1
gamma = 1.0
L = 1
kappa = 10.0

# Set random seed
np.random.seed(0)

# Initialize phi and eta arrays
phi = np.random.randint(Ngrains, size=(Nx, Ny))
eta = {i: np.zeros((Nx, Ny)) for i in range(1, Ngrains + 1)}
for i in range(Nx):
    for j in range(Ny):
        eta[phi[i, j] + 1][i, j] = 1

# Fourier variables
kx = np.fft.fftfreq(Nx, d=dx)
ky = np.fft.fftfreq(Ny, d=dy)
KX, KY = np.meshgrid(kx, ky, indexing='ij')
kpow2 = KX**2 + KY**2

# Time evolution
for step in range(nstep):
    etas_new = np.zeros((Ngrains, Nx, Ny))

    for i in range(1, Ngrains + 1):
        eta_i = eta[i]
        sum_eta_j_sq = np.sum([eta[j]**2 for j in range(1, Ngrains + 1) if j != i], axis=0)

        g = -alpha * eta_i + beta * eta_i**3 + 2 * gamma * eta_i * sum_eta_j_sq
        g_hat = np.fft.fft2(g)
        eta_hat = np.fft.fft2(eta_i)

        numerator = eta_hat - dt * L * g_hat
        denominator = 1 + 4 * np.pi**2 * dt * L * kappa * kpow2
        eta_i_new = numerator / denominator
        etas_new[i-1] = np.real(np.fft.ifft2(eta_i_new))

    # Update eta fields
    for i in range(1, Ngrains + 1):
        eta[i] = etas_new[i - 1]

    # Show plot at every print step
    if step % print_step == 0:
        interaction = np.zeros((Nx, Ny))
        for i in range(1, Ngrains + 1):
            others = sum(eta[j] for j in range(1, Ngrains + 1) if j != i)
            interaction += eta[i] * others
        
        clear_output(wait=True) 
        plt.figure(figsize=(6, 6))
        plt.imshow(interaction, cmap='jet', origin='lower')
        plt.title(f"Step {step}")
        plt.colorbar()
        plt.axis('off')
        plt.show()
        time.sleep(0.1)  
